<a href="https://colab.research.google.com/github/purushottampanchal/temp/blob/main/STT_and_Coqui_TTS_TRAINER_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine tune a VITS model with the Coqui TTS framework using audio samples of your choice.**


In [4]:
#@title gd connect
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Set dataset and trainer output directory (model states saved here, subdirectory of dataset). Set upload_dir and place all audio files (Wav, mp3) in upload directory which will be /content/drive/MyDrive/{upload_dir}

In [8]:
#@title variables
dataset_name = "training-new" #@param {type:"string"}
output_directory = "traineroutput" #@param {type:"string"}
upload_dir = "Uploads" #@param {type:"string"}
upload_dir = "/content/drive/MyDrive/" + upload_dir
#!mkdir $upload_dir

In [ ]:
#@title YT-DLP for downloading audio from Youtube 


!pip install yt-dlp

In [9]:
!ls $upload_dir

training_audio1.wav  training_audio2.wav


In [ ]:
#@title download youtube audio in wav
!yt-dlp -x --audio-format wav -o $upload_dir/training_audio1.wav https://www.youtube.com/watch?v=WQsG8_4JTmw


In [ ]:
!yt-dlp -x --audio-format wav -o $upload_dir/training_audio2.wav https://www.youtube.com/watch?v=TKmf3VKh-Sc


Cleanup previous audio processing

In [11]:
!rm -rf /content/drive/MyDrive/$upload_dir/out
!rm -rf /content/drive/MyDrive/$dataset_name/converted

Download and Build Rnnoise (https://github.com/xiph/rnnoise) and Requirements (Optional if not processing .wav files for a new dataset. Multiple passes of rnnoise over samples may degrade audio quality. You should also preview your output dataset before training to ensure that the processing has not degraded any of your samples.  Occasionally, it will denoise too much of the tail end of phrases)

In [ ]:
!pip install pyloudnorm
!git clone https://github.com/xiph/rnnoise.git
!sudo apt-get install curl autoconf automake libtool python-dev pkg-config sox ffmpeg
%cd /content/rnnoise
!sh autogen.sh
!sh configure
!make clean
!make

Install Sox, Install OpenAI Whisper STT+Translation (https://github.com/openai/whisper)

In [ ]:
%cd /content
!sudo apt install sox
!git clone https://github.com/openai/whisper.git
!pip install git+https://github.com/openai/whisper.git 


Install Coqui TTS (https://github.com/coqui-ai/TTS), espeak-ng phonemeizer (https://github.com/espeak-ng/espeak-ng), download Coqui TTS source and examples from GitHub

In [ ]:
#@title
%cd /content
!sudo apt-get install espeak-ng
!git clone https://github.com/coqui-ai/TTS.git
!pip install TTS
#!tts --list_models

Convert mp3 files in upload_dir to wav as 22050hz mono wav
Place output wav files in directory named 'out'.
Copy any uploaded .wav files to 'out' as 22050hz mono wav.

Make directory 'splits'. Use sox to split wav files into 8 second segments, placed in 'splits'.

In [ ]:
#@title
%cd $upload_dir
!rm -rf $upload_dir/out
!mkdir $upload_dir/out
!find . -name '*.mp3' -exec bash -c 'for f; do ffmpeg -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 out/"${f%.mp3}".wav ; done' _ {} +
!find . -name '*.wav' -exec bash -c 'for f; do ffmpeg -i "$f" -acodec pcm_s16le -ar 22050 -ac 1 out/"${f%.wav}".wav ; done' _ {} +


!ls -al $upload_dir/out
!mkdir $upload_dir/out/splits
%cd $upload_dir/out
!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress silence 1 0.5 0.1% 1 0.5 0.1% : newfile : restart ; done
#alt split method: force splits of 8 seconds, however this will split words. Comment the above with # and remove the # below to change
#!for FILE in *.wav; do sox "$FILE" splits/"$FILE" --show-progress trim 0 8 : restart ; done
%cd $upload_dir/out/splits
!find . -name "*.wav" -type f -size -15k -delete

(Optional) Run rnnoise, processed clips will be saved to your Google Drive in a folder named 'converted'. Files converted to 22khz mono for VITS model fine tuning.

Original author unknown, but thank you for posting this helpful code.

In [ ]:
#@title
orig_wavs= upload_dir + "/out/splits"
from pathlib import Path
import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import sys
import glob
rnn = "/content/rnnoise/examples/rnnoise_demo"
paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
for filepath in paths:
  base = os.path.basename(filepath)
  tp_s = "/content/drive/MyDrive/" + dataset_name + "/" + "converted" + "/"
  tf_s = "/content/drive/MyDrive/" + dataset_name + "/" + "converted" + "/" + base
  target_path = Path(tp_s)
  target_file = Path(tf_s)
  print("From: " + str(filepath))
  print("To: " + str(target_file))
	
# Stereo to Mono; upsample to 48000Hz
# added -G to fix gain, -v 0.95
  subprocess.run(["sox", "-G", "-v", "0.95", filepath, "48k.wav", "remix", "-", "rate", "48000"])
  subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
  subprocess.run(["/content/rnnoise/examples/rnnoise_demo", "temp.raw", "rnn.raw"]) # apply rnnoise
  subprocess.run(["sox", "-G", "-v", "0.95", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav

  subprocess.run(["mkdir", "-p", str(target_path)])
  subprocess.run(["sox", "rnn.wav", str(target_file), "remix", "-", "highpass", "100", "lowpass", "7000", "rate", "22050"]) # apply high/low pass filter and change sr to 22050Hz
  data, rate = sf.read(target_file)

# peak normalize audio to -1 dB
  peak_normalized_audio = pyln.normalize.peak(data, -1.0)

# measure the loudness first
  meter = pyln.Meter(rate) # create BS.1770 meter
  loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
  loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
  sf.write(target_file, data=loudness_normalized_audio, samplerate=22050)
  print("")
%cd /content/drive/MyDrive/"$dataset_name"/converted/
!find . -name "*.wav" -type f -size -15k -delete


In [ ]:
#@title Copy wav files to dataset wav directory
!mkdir /content/drive/MyDrive/$dataset_name/wavs
!cp /content/drive/MyDrive/$dataset_name/converted/*.wav /content/drive/MyDrive/$dataset_name/wavs
!ls -al /content/drive/MyDrive/$dataset_name/wavs

In [ ]:
#@title Run Whisper on generated audio clips, generate transcript named metadata.csv in LJSpeech format in the dataset directory.
import whisper
import os, os.path
import glob
import pandas as pd

from pathlib import Path

wavs = '/content/drive/MyDrive/'+dataset_name+'/wavs'

model = whisper.load_model("medium.en")
paths = glob.glob(os.path.join(wavs, '*.wav'))
print(len(paths))

all_filenames = []
transcript_text = []
with open('/content/drive/MyDrive/'+dataset_name+'/metadata.csv', 'w', encoding='utf-8') as outfile:
	for filepath in paths:
		base = os.path.basename(filepath)
		all_filenames.append(base)
	for filepath in paths:
		result = model.transcribe(filepath)
		output = result["text"].lstrip()
		output = output.replace("\n","")
		thefile = str(os.path.basename(filepath).lstrip(".")).rsplit(".")[0]
		outfile.write(thefile + '|' + output + '|' + output + '\n')
		print(thefile + '|' + output + '|' + output + '\n')


In [ ]:
#@title Display dataset
!cat /content/drive/MyDrive/$dataset_name/metadata.csv

Download VITS model and Generate Sample Wav File to /content/ljspeech-vits.wav  This will be deleted when your Colab session is closed.

In [ ]:
!tts --text "I am the very model of a modern Major General" --model_name "tts_models/en/ljspeech/vits" --out_path /content/ljspeech-vits.wav

Save the training script with your dataset name, and output dir set. File will be saved in your Google drive dataset folder as train_vits.py

In [31]:
#@title
code = """import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

#output_path = os.path.dirname(os.path.abspath(__file__))
##########################################
#Change this to your dataset directory
##########################################
output_path = "/content/drive/MyDrive/"""
code = code + dataset_name + "/" + output_directory + "/" + "\""

code=code + """
dataset_config = BaseDatasetConfig(
##########################################
#Change this to your dataset directory
##########################################
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "/content/drive/MyDrive/"""
code = code + dataset_name
code=code + """")

)
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech",
    batch_size=8,
    eval_batch_size=8,
    batch_group_size=4,
#    num_loader_workers=8,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100000,
    save_step=1000,
	  save_checkpoints=True,
	  save_n_checkpoints=3,
	  save_best_after=1000,
    #text_cleaner="english_cleaners",
    text_cleaner="multilingual_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
)

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

# init the trainer and begin
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
trainer.fit()
"""
#print(code)
myFile = open("/content/drive/MyDrive/" + dataset_name + "/train_vits.py", 'w')

myFile.write(code)
myFile.close()
%ls -al /content/drive/MyDrive/$dataset_name/train_vits.py

-rw------- 1 root root 2995 May  5 06:16 /content/drive/MyDrive/training-new/train_vits.py


Fine Tune VITS model

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python /content/drive/MyDrive/$dataset_name/train_vits.py --restore_path /root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth --config_file /root/.local/share/tts/tts_models--en--ljspeech--vits/config.json

**Resume a fine tuning session**

Run the next cell to list all of the saved run direcories.  Copy select the run you want to resume, ctrl+C to copy the name. Paste in the next cell and run it.

In [3]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory

phoneme_cache
vits_ljspeech-May-05-2023_06+22AM-0000000
vits_ljspeech-May-05-2023_06+33AM-0000000
vits_ljspeech-May-05-2023_08+39AM-0000000
vits_ljspeech-May-05-2023_10+42AM-0000000
vits_ljspeech-May-05-2023_10+51AM-0000000


In [22]:
run_name = "vits_ljspeech-May-05-2023_06+33AM-0000000" #@param {type:"string"}

Run this cell to list all of the checkpoints in the saved run, then put the checkpoint name in the next cell, and run it.

In [23]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory/$run_name

best_model_1003474.pth	config.json
best_model.pth		events.out.tfevents.1683268381.8f816e09b6f9
checkpoint_1002000.pth	trainer_0_log.txt
checkpoint_1003000.pth	train_vits.py
checkpoint_1004000.pth


In [29]:
model_checkpoint = "best_model_1003474.pth" #@param {type:"string"}

Run the next cell to restore a previous fine tuning session using your dataset name, trainer output directory, and model checkpoint set

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python /content/drive/MyDrive/$dataset_name/train_vits.py --restore_path /content/drive/MyDrive/$dataset_name/$output_directory/$run_name/$model_checkpoint --config_path /content/drive/MyDrive/$dataset_name/$run_name/config.json

View Memory Usage

In [ ]:
!nvidia-smi

In [ ]:
#@title Load Tensorboard
import torch 
%load_ext tensorboard

In [ ]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory/

In [16]:
!kill 1349

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/$dataset_name/$output_directory/ --port 8008